# Moving Beyond the Basics

In [51]:
import pandas as pd
reviews = pd.read_csv('winemag-data_first150k.csv')

## Summary Functions

There are a number of built in function that can allow us to see an overview of data in a column (or row) similar to the describe() and head() functions

In [52]:
# Gives the mean value of a int column
reviews.points.mean()

# Returns a list of unique values in a column
reviews.country.unique()

# Returns a list of the unique values and how many of each there are
reviews.country.value_counts()

# Returns the index of the first occurence of the max value in a series
reviews.price.idxmax()

34920

## Maps

Maps are a mathemetical concept that takes one set of values and 'maps' it onto another set of values.
In data science they are a common tool to transform data to the format we want


The first and simpler mapping function in pandas is the .map() method.

    - map() expects a single value from the Series and returns a transformed version of the value (you can use a lambda function or a custom function for the transformation).
    
    - apply() is the equivalent method to transform a whole DataFrame by calling a custom method on each row.

This is how we can apply a custom function to a column of data.
Both map() and apply() return a new transformed version of the Series / DataFrame respectively and don't edit in place.

In [53]:
# Takes the points series and shifts the mean to be 0
review_points_mean = reviews.points.mean()

reviews.points.map(lambda p: p - review_points_mean)

# Function that shifts the points so the mean is 0
def remean_points(row):
    row.points = row.points - review_points_mean
    return row

reviews.apply(remean_points, axis='columns')
# If we had called reviews.apply() with axis='index', then instead of passing a function to 
# transform each row, we would need to give a function to transform each column.

# Maintains original values since we did not assign output of .apply()
reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


Lots of common functions are built into pandas already such as the basic mathematical operators + - * and /.

Pandas will understand what we are trying to do when we use these operators on series of equal length. Very versatile.

These basic operators are FASTER than map() and apply() but not as FLEXIBLE.

In [54]:
# Faster way to remean
reviews_points_mean = reviews.points.mean()
reviews.points - reviews_points_mean

# Create new Series based on other series
reviews.country + ' - ' + reviews.region_1

0                  US - Napa Valley
1                      Spain - Toro
2               US - Knights Valley
3            US - Willamette Valley
4                   France - Bandol
                    ...            
150925    Italy - Fiano di Avellino
150926           France - Champagne
150927    Italy - Fiano di Avellino
150928           France - Champagne
150929           Italy - Alto Adige
Length: 150930, dtype: object

## Grouping

groupby() is a function that groups data together based on a given column. value_counts() is an example of a shortcut to using groupby().

reviews.points.value_counts() is the same as reviews.groupby('points').points.count()


In [55]:
reviews.points.value_counts()

reviews.groupby('points').points.count()

points
80       898
81      1502
82      4041
83      6048
84     10708
85     12411
86     15573
87     20747
88     17871
89     12921
90     15973
91     10536
92      9241
93      6017
94      3462
95      1716
96       695
97       365
98       131
99        50
100       24
Name: points, dtype: int64

Each group that is generated is like a little slice of data with matching values. This method returns a new dataframe for each mathcing value that can be accessed directly using the apply() method.

For more fine control you can also group by multiple columns.



In [56]:
# Retrieves the first wine by each winery in the data set
reviews.groupby('winery').apply(lambda df: df.designation.iloc[0])

# Groups the data by country and province then finds the first instance of wine with the most points
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

C:\Users\elias\AppData\Local\Temp\ipykernel_27928\1572352964.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews.groupby('winery').apply(lambda df: df.designation.iloc[0])
C:\Users\elias\AppData\Local\Temp\ipykernel_27928\1572352964.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])


Unnamed: 0    country  \
country   province                                  
Albania   Mirditë                 4642    Albania   
Argentina Mendoza Province       65331  Argentina   
          Other                  10619  Argentina   
Australia Australia Other        68251  Australia   
          New South Wales        54205  Australia   
...                                ...        ...   
Uruguay   Juanico                 3160    Uruguay   
          Montevideo              3164    Uruguay   
          Progreso                6541    Uruguay   
          San Jose               70157    Uruguay   
          Uruguay               132482    Uruguay   

                                                                  description  \
country   province                                                              
Albania   Mirditë           This garnet-colored wine made from 100% Kallme...   
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
Australia Australia Other   This big wine presents a sophisticated bouquet...   
          New South Wales   This wine's deep brassy color suggests honey, ...   
...                                                                       ...   
Uruguay   Juanico           This mature Bordeaux-style blend is earthy on ...   
          Montevideo        Bouza ranks as one of Uruguay's top wineries, ...   
          Progreso          Blackberry and plum aromas come with wood-smok...   
          San Jose          While this ranks as one of the best Uruguayan ...   
          Uruguay           They call it Special Barrel, and one sniff tel...   

                                                        designation  points  \
country   province                                                            
Albania   Mirditë                                               NaN      88   
Argentina Mendoza Province                         Nicasia Vineyard      97   
          Other                                             Reserva      95   
Australia Australia Other                                  Yattarna      92   
          New South Wales                        Noble One Botrytis      93   
...                                                             ...     ...   
Uruguay   Juanico                  Preludio Barrel Select Lote N 77      90   
          Montevideo        Monte Vide Eu Tannat-Merlot-Tempranillo      90   
          Progreso                                              RPF      89   
          San Jose                 El Preciado Premier Gran Reserva      89   
          Uruguay                                    Special Barrel      89   

                            price          province                 region_1  \
country   province                                                             
Albania   Mirditë            20.0           Mirditë                      NaN   
Argentina Mendoza Province  120.0  Mendoza Province                  Mendoza   
          Other              90.0             Other                    Salta   
Australia Australia Other    65.0   Australia Other  South Eastern Australia   
          New South Wales    32.0   New South Wales          New South Wales   
...                           ...               ...                      ...   
Uruguay   Juanico            45.0           Juanico                      NaN   
          Montevideo         57.0        Montevideo                      NaN   
          Progreso           23.0          Progreso                      NaN   
          San Jose           60.0          San Jose                      NaN   
          Uruguay            50.0           Uruguay                      NaN   

                           region_2     variety                winery  
country   province                                                     
Albania   Mirditë               NaN     Kallmet                Arbëri  

Another gourpby() method is agg() which lets us run a bunch of functions on the dataframe simultaneously.

In [57]:
# A simple statistical summary of the data
reviews.groupby(['country']).price.agg([len, min, max])

C:\Users\elias\AppData\Local\Temp\ipykernel_27928\4273679212.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  reviews.groupby(['country']).price.agg([len, min, max])
C:\Users\elias\AppData\Local\Temp\ipykernel_27928\4273679212.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  reviews.groupby(['country']).price.agg([len, min, max])


,len,min,max
country,,,
Albania,2,20.0,20.0
Argentina,5631,4.0,250.0
Australia,4957,5.0,850.0
Austria,3057,8.0,1100.0
Bosnia and Herzegovina,4,12.0,13.0
Brazil,25,11.0,35.0
Bulgaria,77,7.0,28.0
Canada,196,12.0,145.0
Chile,5816,5.0,400.0


## Multi-indexing

In the previous examples we have been working with DataFrames that only have one index. Using groupby can often result in DataFrames with multiple indexes (they can also be set manually).

In [58]:
mi_reviewed = reviews.groupby(['country', 'province']).description.agg([len])

type(mi_reviewed.index)

pandas.core.indexes.multi.MultiIndex

A common multiIndex method is reset_index() which results in breaking the df back out into a single index and adding the other indeces back to the regular columns

In [59]:
mi_reviewed.reset_index()

,country,province,len
0,Albania,Mirditë,2
1,Argentina,Mendoza Province,4742
2,Argentina,Other,889
3,Australia,Australia Other,553
4,Australia,New South Wales,246
...,...,...,...
450,Uruguay,Juanico,19
451,Uruguay,Montevideo,3
452,Uruguay,Progreso,5
453,Uruguay,San Jose,15


## Sorting

    - sort_values() can be used to sort a dataframe by data in a particular column
    - Arguments includ:
        - by='column name' (str or list of column name(s))
        - ascending = bool
    - sort_index() has the same arguments as sort_values(), can return a df to it's original order

In [60]:
mi_reviewed = mi_reviewed.reset_index()

mi_reviewed.sort_values(by='len')

mi_reviewed.sort_values(by='len', ascending=False)

mi_reviewed.sort_values(by=['len', 'country'])

,country,province,len
17,Austria,Langenlois,1
42,Brazil,Serra do Sudeste,1
43,Brazil,Vale Trentino,1
49,Bulgaria,Rose Valley,1
64,Chile,Casablanca-Curicó Valley,1
...,...,...,...
407,Spain,Northern Spain,4892
122,France,Bordeaux,6111
242,Italy,Tuscany,7281
442,US,Washington,9750


## Dtypes

the data type for a column is known as its dtype. each column has a dtype property and can be accessed using pythons attribute notation.

    - reviews.price.dtype
    - dtype('float64')

dtypes can also be used on a DataFrame to return the a series of the dtypes of all columns.
Columns with only strings are given teh 'object' type.

Data types are mutable when it makes sense and can be changed using the astype() method

A DataFrame index also has a dtype

In [64]:
# Display dtype of a column
reviews.price.dtype

# Display dtypes of a df
reviews.dtypes

# Change dtype from one to another
reviews.points.astype('float64') 

# View the index dtype
reviews.index.dtype

dtype('int64')

## Missing Data

Finding and dealing with missing data or NaN values is very important in pandas. NaN values will often raise errors when performing large scale operations and should be changed to something our program can recognize and we can deal with.

NaN values always have a dtype of 'float64'.

To select NaN entries:
    - pd.isnull(df.column): returns a dataframe where all entires in the given column are NaN
    - pd.notnull(df.column): returns the opposite, no NaN entries in that column.

In [ ]:
# Get null entries
reviews[pd.isnull(reviews.country)]

# Get not null entries
reviews[pd.notnull(reviews.country)]

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
1133,1133,NaN,Delicate white flowers and a spin of lemon pee...,Askitikos,90,17.0,NaN,NaN,NaN,Assyrtiko,Tsililis
1440,1440,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Red Blend,Büyülübağ
68226,68226,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
113016,113016,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
135696,135696,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas


Replacing the missing values is a common operation.
    - fillna(value_to_replace_na_with)

    - replace('value to replace', 'replacement'): another useful method for editing data

In [78]:
# Fill in NaN values with "unknown"
reviews.country.fillna('Unknown')

# Replace prices when a company changes their prices
reviews.price.replace(4.0, 5.0)

0         235.0
1         110.0
2          90.0
3          65.0
4          66.0
          ...  
150925     20.0
150926     27.0
150927     20.0
150928     52.0
150929     15.0
Name: price, Length: 150930, dtype: float64